In [1]:
# 🧪 SVM Example: Classifying Synthetic Data

# ------------------------------------------------------------
# 📦 Import libraries
# ------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from svm import SVMModel  # 🧠 Our custom SVM wrapper (uses scikit-learn)

In [ ]:
# ------------------------------------------------------------
# 🎲 Step 1: Generate synthetic classification data
# ------------------------------------------------------------

# ❓ Why: This step creates fake data to simulate a real-world binary classification task.
# It's useful for experimenting with algorithms without needing a real dataset.

# 💡 Fun fact: Synthetic data is widely used in ML to test models before applying them to real problems!

# ------------------------------------------------------------
# ✅ Step A: Set the random seed for reproducibility
# ------------------------------------------------------------
np.random.seed(42)
# Setting a seed makes sure we get the same "random" results each time we run the code.
# Great for debugging or sharing results with others.

# ------------------------------------------------------------
# 🧊 Step B: Generate data points for Class 0
# ------------------------------------------------------------
# The covariance matrix controls how data points are spread and tilted in space.
# The diagonal values represent variance along the x and y axes (how wide the data spreads),
# while the off-diagonal values represent covariance (how much x and y move together).
# A positive covariance tilts the data upward (x↑ → y↑), a negative one tilts it downward (x↑ → y↓),
# and zero means no directional relationship. Adjusting this matrix changes the shape and orientation
# of the data cloud, making it a powerful tool for simulating realistic patterns in synthetic datasets.

class0 = np.random.multivariate_normal(
    mean=[2, 2],                # 📍 Center of the class 0 cluster (x=2, y=2)
    cov=[[1, 0.5], [0.5, 1]],   # 🔄 Covariance matrix (controls shape/orientation of the spread)
    size=50                     # 🎯 Number of data points to generate
)
# This creates 50 points clustered around (2,2) with some spread/tilt.

# ------------------------------------------------------------
# 🔥 Step C: Generate data points for Class 1
# ------------------------------------------------------------
class1 = np.random.multivariate_normal(
    mean=[5, 5],                # 📍 Center of the class 1 cluster (x=5, y=5)
    cov=[[1, -0.5], [-0.5, 1]], # 🔄 Covariance gives it a different spread/direction than class0
    size=50                     # 🎯 Also generating 50 points here
)
# Now we have a second cluster of 50 points around (5,5), forming our second class.

# ------------------------------------------------------------
# 🧩 Step D: Combine the feature data from both classes
# ------------------------------------------------------------
X = np.vstack((class0, class1))
# 🧱 `vstack` vertically stacks the two arrays (50 + 50 = 100 rows)
# Final shape of X: (100, 2) — 100 samples, each with 2 features (x and y coordinates)

# ------------------------------------------------------------
# 🏷️ Step E: Create the labels for the classes
# ------------------------------------------------------------
y = np.array([0]*50 + [1]*50)
# 🧠 First 50 labels are 0 (class0), next 50 are 1 (class1)
# Final shape of y: (100,) — 100 labels total
# ------------------------------------------------------------

# 📊 Step F: Visualize the synthetic data
combined_dataset = np.column_stack((y, X))
print("Combined dataset: \n", combined_dataset)

# ✅ Now, X and y together form a labeled dataset you can use for training/testing classifiers


In [3]:
# ------------------------------------------------------------
# ✂️ Step 2: Train/test split
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# ------------------------------------------------------------
# 🧠 Step 3: Train SVM model
# ------------------------------------------------------------
# The SVM kernel defines how it draws the decision boundary between classes.
# 🔹 "linear" — Best for data that’s linearly separable (straight line boundary).
# 🔹 "rbf" (Radial Basis Function) — Handles complex, curvy boundaries. Great default!
# 🔹 "poly" — Uses polynomial functions (quadratic, cubic, etc.). Good for medium complexity.
# 🔹 "sigmoid" — Mimics a neural net activation; rarely used in practice.
# 💡 Tip: Try different kernels when accuracy is low — each one sees the data differently!
model = SVMModel(kernel="linear")  
model.train(X_train, y_train)



In [ ]:
# ------------------------------------------------------------
# 🔍 Step 4: Make predictions and evaluate
# ------------------------------------------------------------
y_pred = model.predict(X_test)

# Print evaluation results
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📊 Classification Report:\n", classification_report(y_test, y_pred))



In [ ]:
# ------------------------------------------------------------
# 📈 Step 5: Visualize decision boundary
# ------------------------------------------------------------

def plot_decision_boundary(model, X, y):
    """
    Visualizes the decision boundary of a trained SVM model on 2D data.

    Parameters:
    - model: A trained classifier with a .predict() method (e.g., SVM).
    - X: Feature matrix of shape (n_samples, 2), where each row is [x1, x2].
    - y: Label array of shape (n_samples,), with class labels (e.g., 0 or 1).
    """

    # -------------------------
    # 🧱 Step 1: Create mesh grid
    # -------------------------

    # This step builds a "canvas" of 2D points that we'll color based on the model's predictions.

    # Set the step size for the meshgrid.
    # Smaller = smoother boundary, but more computation.
    h = 0.02

    # Extract x-axis range (first feature) from the dataset:
    # X[:, 0] → All rows of column 0 (feature 1)
    x_feature_column = X[:, 0]
    x_min = x_feature_column.min() - 1  # Minimum x value minus padding
    x_max = x_feature_column.max() + 1  # Maximum x value plus padding

    # Extract y-axis range (second feature) from the dataset:
    # X[:, 1] → All rows of column 1 (feature 2)
    y_feature_column = X[:, 1]
    y_min = y_feature_column.min() - 1  # Minimum y value minus padding
    y_max = y_feature_column.max() + 1  # Maximum y value plus padding

    # Generate evenly spaced values across the x and y ranges
    x_values = np.arange(x_min, x_max, h)
    y_values = np.arange(y_min, y_max, h)

    # Create a meshgrid from these x and y values:
    #   xx and yy are 2D arrays that together represent all (x, y) coordinate pairs
    #   This will create a grid of points we can classify
    xx, yy = np.meshgrid(x_values, y_values)

    # -------------------------------------------------------------
    # 🤖 Step 2: Predict labels for each point in the mesh grid
    # -------------------------------------------------------------

    # Flatten xx and yy and stack them column-wise using np.c_ to create input feature vectors
    # np.c_ → Concatenates arrays column-wise (axis=1)
    # Example:
    #   xx.ravel() = [x1, x2, x3, ...]
    #   yy.ravel() = [y1, y2, y3, ...]
    #   np.c_[xx.ravel(), yy.ravel()] = [[x1, y1], [x2, y2], [x3, y3], ...]
    grid_points = np.c_[xx.ravel(), yy.ravel()]  # Shape: (num_grid_points, 2)

    # Use the model to predict class labels (0 or 1) for each grid point
    Z = model.predict(grid_points)  # Output shape: (num_grid_points,)

    # Reshape the prediction array to match the meshgrid shape (2D)
    Z = Z.reshape(xx.shape)

    # ---------------------------------------------
    # 🎨 Step 3: Plot the decision boundary + points
    # ---------------------------------------------

    # Start a new figure with a defined size
    plt.figure(figsize=(8, 6))

    # Draw filled contours based on predictions (Z)
    # This colors the background based on the predicted class at each (x, y) point
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)

    # Plot the actual data points
    # X[:, 0] → x-coordinates
    # X[:, 1] → y-coordinates
    # y       → color for each point based on its class label
    plt.scatter(
        X[:, 0],           # Feature 1 (x-axis)
        X[:, 1],           # Feature 2 (y-axis)
        c=y,               # Color points by their label (0 or 1)
        cmap=plt.cm.coolwarm,
        edgecolors="k"     # Outline each point with a black border
    )

    # Add plot details
    plt.title("SVM Decision Boundary")    # Title
    plt.xlabel("Feature 1")               # x-axis label
    plt.ylabel("Feature 2")               # y-axis label
    plt.grid(True)                        # Show grid
    plt.tight_layout()                    # Fit layout cleanly

    # Display the final plot
    plt.show()


# 🧪 Call the function with your trained model and data
plot_decision_boundary(model, X, y)

